In [ ]:
import h5py
path = "/Volumes/HEIU/data/bulkice_train.hdf5"
print(path)
f = h5py.File(path, "r")


In [ ]:
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
plt.imshow(make_grid(afm.transpose(1,0)).permute(1,2,0))

In [ ]:
f['T180_13015'].afm

In [ ]:
from dataset.dataset import z_sampler
z_sampler(10, 4, True)

In [ ]:
import torch
from torch import nn
import torchvision
import matplotlib.pyplot as plt
import time
from dataset import transform as tf

pics = torch.ones(1, 8, 100, 100) * 0.9
pics[..., 50:, 0:50] = 0.1
pics[..., :50, 50:] = 0.1

transform = nn.Sequential(
    tf.PixelShift(),
    tf.Cutout(),
    tf.ColorJitter(),
    tf.Noisy(),
    tf.Blur(),
)
t = time.time()
for i in range(1):
    pics = transform(pics)

print(time.time() - t)


T = torchvision.utils.make_grid(pics.transpose(1,0),pad_value = 0.5)
plt.imshow(T.permute(1,2,0))


In [ ]:
import torch
torch.arange(10)

In [ ]:
a = torch.ones(100, 100)
a[10:20, 10:20]

In [ ]:
import torch
a = torch.rand(100, 100, 3)
b = torch.rand(100, 100) > 0.5
a[b]

In [ ]:
import torch
a = torch.ones((50, 50, 5))
c = torch.rand((50, 50, 5, 3))
mask = a.nonzero()
print(mask.T.shape)
a[tuple(mask.T)]
(c[tuple(mask.T)] + mask) / (50, 50, 5)

In [ ]:
import torch
from utils import poscar
box_cls = torch.randint(0, 3, (3, 50, 50), dtype=torch.long)
box_off = torch.rand((3, 50, 50, 3))
print(poscar.boxncls2pos_torch(box_cls, box_off))
print(poscar.boxncls2pos_np(box_cls.numpy(), box_off.numpy()))

In [ ]:
import torch
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
pred_types, pred_pos = poscar.targ2pred(label_type, label_pos)
a = torch.jit.script(Analyser())
pred_types = pred_types.unsqueeze(0).repeat((10, 1, 1, 1, 1))
pred_pos  = pred_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))
label_type = label_type.unsqueeze(0).repeat((10, 1, 1, 1))
label_pos = label_pos.unsqueeze(0).repeat((10, 1, 1, 1, 1))

In [ ]:
import time
t = time.time()
for i in range(1000):
    a(pred_types, pred_pos, label_type, label_pos)
print(time.time() - t)

In [ ]:
a = torch.rand((100, 100, 3))
mask = torch.nonzero(a, as_tuple=True)
a[mask[0], mask[1], mask[2]]

In [ ]:
import time
import torch
from utils.metrics import ConfusionMatrixCounter
cm = torch.randint(0, 100, (1, 2, 1, 3)).numpy()
t = time.time()
ConfusionMatrixCounter._count(cm)

In [ ]:
import torch
from model import unet_onehot
m = unet_onehot()
img = torch.rand((1, 1, 10, 100, 100))
m(img)

In [ ]:
import torch
import numpy as np
from utils.metrics import Analyser
from utils import poscar
from dataset import AFMDataset
path = "/Volumes/HEIU/data/bulkice_train.hdf5"

dts = AFMDataset(path)
fn, afm, label_type, label_pos = dts[0]
print(fn)
typ, pos, _ = poscar.boxncls2pos_torch(label_type, label_pos)
typ = typ.numpy()
pos = pos.numpy()
print(len(pos))
pos2 = np.concatenate([pos[typ == i] for i in range(1,3)])
poscar.save("test.poscar", [3.0, 25.0, 25.0], ['O', 'H'], [48, 96], pos2)

In [ ]:
import numpy as np
a = ["1", "2", "3"]
np.array(a, dtype=np.float32)

In [ ]:
int("9246.0")

In [ ]:
import numpy as np
from numba import jit

@jit(nopython=True)
def getWaterRotate(tag: np.ndarray, inv_ref: np.ndarray) -> np.ndarray:
    O, H1, H2 = tag
    H1, H2 = H1 - O, H2 - O
    H1 = H1 / 0.9572
    H2 = H2 / 0.9572
    H3 = np.cross(H1, H2)
    if H3[2] < 0:
        tag[0] = H2
        tag[1] = H1
        tag[2] = -H3
    else:
        tag[0] = H1
        tag[1] = H2
        tag[2] = H3
    return inv_ref @ tag

ang = 104.52 / 180 * np.pi
v = np.array([0, 0, 1])
u = np.array([np.sin(ang), 0, np.cos(ang)])
r = np.cross(v, u)
ref = np.asarray([v, u, r])
invref = np.linalg.inv(ref)

pos = np.asarray([[6.20233009,  6.80328286,  0.71540000], [6.72382009,  6.43174286,  0.00390000], [5.46304009, 6.20268286,  0.81010000]])
#pos = np.asarray([[14.10287009, 18.97821286,  0.47590000], [14.35087009, 19.79591286,  0.04450000], [13.24817009, 19.16231286,  0.86570000]])
import time
t = time.time()
for i in range(1000):
    R = getWaterRotate(pos.copy(), invref)
print(time.time() - t)

print(R,np.linalg.det(R))
print(R @ R.T)
print(R @ ref)

In [ ]:
import torch
from model import VAEunet
import utils
net = VAEunet(channel_mult = (1, 2, 3, 4))
temp = torch.rand((1, 1))
print("\n".join(utils.model.model_structure(net)))
image = torch.rand((1, 10, 4,25, 25))
mu, nu, x = net(image, temp)

In [ ]:
from dataset import AFMGenDataset
dts = AFMGenDataset("/Volumes/HEIU/data/ice_8_4A/ice_8_4A_train.hdf5")
name, box4a, box8a = dts[0]

In [ ]:
print(len(box4a[...,0].nonzero()))
print(len(box8a[...,0].nonzero()))

In [ ]:
import torch
a = torch.zeros(10)
a[(0,1,2),] = 1
print(a)

In [ ]:
from utils.functional import cdist2, sort, nms_mask

In [ ]:
import numpy as np
import torch
import time
u = np.random.rand(10000, 3)
v = np.random.rand(10000, 3)
t = time.time()
for i in range(100):
    out_n = cdist2(u)
print(time.time() - t)
u, v = torch.from_numpy(u), torch.from_numpy(v)
t = time.time()
for i in range(100):
    out_t = torch.cdist(u, u, 2)
print(time.time() - t)
print(out_n, out_t)

In [ ]:
import numpy as np
sort(np.random.rand(100))

In [ ]:
import numpy as np
import time
from utils.functional import nms_mask
a = np.random.rand(1000, 3) * 1
t = time.time()
iu = np.triu_indices(a.shape[0])
for i in range(100):
    mask = nms_mask(a, 0.1)
print(time.time() - t)
# a = np.triu(np.where(a > 0.5, np.inf, a))
# np.unravel_index(np.argpartition(a, 100 * 99 // 2, axis=None), a.shape)

import torch
from torch import Tensor

def unravel_index(index, shape):
    out = []
    for dim in reversed(shape):
        out.append(index % dim)
        index = index // dim
    return tuple(reversed(out))

def argmatch(A: Tensor, B: Tensor, cutoff: float) -> tuple[Tensor]:
    dis = torch.cdist(A, B)
    dis = dis < cutoff
    
    
    return out

@torch.jit.script
def nms_mask(pos: Tensor, cutoff: float) -> Tensor:
    DIS = torch.cdist(pos, pos)
    DIS = DIS < cutoff
    DIS = (torch.triu(DIS, diagonal= 1)).float()
    args = torch.ones(pos.shape[0], dtype = torch.bool, device = pos.device)
    while True:
        N = pos.shape[0]
        restrain_tensor = DIS.sum(0)
        restrain_tensor -= ((restrain_tensor != 0).float() @ DIS)
        SELECT = restrain_tensor == 0
        DIS = DIS[SELECT][:, SELECT]
        pos = pos[SELECT]
        args[args.clone()] = SELECT
        if N == pos.shape[0]:
            break
    return args



b = torch.from_numpy(a)
t = time.time()
for i in range(100):
    out = nms(b, 0.1)
print(time.time() - t)
print(out.shape, mask.shape)
print(out.sum(), mask.sum())
pos = torch.cdist(b[out], b[out])
pos = torch.triu(pos < 0.1, 1)
print(pos.sum())
# print(out.numpy()[:50])
# print(a[mask][:50])
print(np.allclose(out.numpy(), mask))


In [208]:
def argmatch(pred: Tensor, targ: Tensor, cutoff: float) -> tuple[Tensor]:
    dis = torch.cdist(targ, pred)
    dis = (dis < cutoff).nonzero()
    dis = dis[:, (1, 0)]
    unique, idx, counts = torch.unique(dis[...,1], sorted=True, return_inverse=True, return_counts=True)
    ind_sorted = torch.argsort(idx, stable=True)
    cum_sum = counts.cumsum(0)
    cum_sum = torch.cat((torch.tensor([0]), cum_sum[:-1]))
    first_indicies = ind_sorted[cum_sum]
    dis = dis[first_indicies]
    return dis

import torch
b = torch.rand(1000, 3)
c = torch.rand(100, 3)
t = time.time()
for i in range(1000):
    reg = argmatch(b, c, 0.1)
print(time.time() - t)
reg = argmatch(b, c, 0.1).T
print(reg)
print(reg[0].unique().shape, reg[0].shape)


0.3666548728942871
tensor([[100, 143, 156, 395, 336, 209, 350, 433, 100, 157,   4, 402,  73,  37,
          47, 141, 237, 266, 132,  97, 354,   0, 736, 162, 187,  58,  77, 385,
         142, 100, 185, 187, 156, 189, 259, 268, 163,   0, 802, 541, 829,   4,
          28,  90, 471, 175, 220,  13,  29, 139, 159,  97,  51,  51, 649, 215,
          60, 472,  24, 140, 215,  90, 118, 200, 664, 180,  60, 984,  93,  83,
         381,  58,  40, 206, 592,  64, 237,  51, 235, 418,  22, 126, 215, 615,
         163, 176,  88, 120, 329,  70, 911, 190, 101, 143, 403],
        [  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,  15,
          16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
          30,  31,  32,  33,  34,  36,  37,  38,  39,  40,  41,  42,  43,  44,
          45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
          59,  60,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
          74,  75,  76,  77,  78,  79,  80,  81